# <h1 align="center">Introduction to Bayesian Optimization</h1>
<h3 align="right">Linjian Li</h3>
<h3 align="right">2018-12-22</h3>


## Some Application Examples $f(x)$
- Hyperparameter optimization: $f$ is training loss and $x$ are some hard-to-set hyperparameters
- Recommending ads: $f$ is the expected number of clicks and $x$ are ads

We want to optimize $f(x)$, but $f$ may be expensive to evaluate. So, we need to make the number of  evaluation of f as less as possible. We can just take random guesses.
<img src="guess optimum.png">


## Feature
- Global optimization
- For Black-box functions
- Does not require derivatives



## Gaussian Process
- a stochastic process
- any finite subcollection of random variables has a multivariate Gaussian distribution
- **mean function $m(\cdot)$** and **covariance function (kernal) $k(\cdot, \cdot)$**
    - In general, $m(\cdot)$ can be any real-valued function is acceptable. May be just some simple functions that draw a smooth curve connecting all the observed points.
    - $k(\cdot, \cdot)$ is positive semidefinite.
    <img src="usual covariance functions.png">

### Posterior

$$
\begin{pmatrix}\mathbf{y} \\ \mathbf{f}_*\end{pmatrix} \sim \mathcal{N}
\left(\begin{pmatrix}\boldsymbol{\mu} \\ \boldsymbol{\mu_*} \end{pmatrix},
\begin{pmatrix}\mathbf{K}_y & \mathbf{K}_* \\ \mathbf{K}_*^T & \mathbf{K}_{**}\end{pmatrix}
\right)\tag{1}\label{eq1}
$$

$$
\begin{align*}
p(\mathbf{f}_* \lvert \mathbf{X}_*,\mathbf{X},\mathbf{y}) 
&= \int{p(\mathbf{f}_* \lvert \mathbf{X}_*,\mathbf{f})p(\mathbf{f} \lvert \mathbf{X},\mathbf{y})}\ d\mathbf{f} \\ 
&= \mathcal{N}(\mathbf{f}_* \lvert \boldsymbol{\mu}_*, \boldsymbol{\Sigma}_*)
\end{align*} \tag{2}\label{eq2}
$$

$$
\begin{align*}
\boldsymbol{\mu_*} &= \mathbf{K}_*^T \mathbf{K}_*^{-1} \mathbf{y} \tag{4}\label{eq4}\\
\boldsymbol{\Sigma_*} &= \mathbf{K}_{**} - \mathbf{K}_*^T \mathbf{K}_*^{-1} \mathbf{K}_* \tag{5}\label{eq5}
\end{align*}
$$


<img src="GP.png">

The gray dash line is the invisible true function. <br/>
The green solid line is the **mean** our guess of the true function. <br/>
The green translucent green areas are the **standard deviation** of our guess.

So, which point to be sampled next?
- Exploitation-exploration (mean-variance) trade-off
    - Areas with high confidence of yielding better values (Exploitation)
    - Areas with high uncertainty (Exploration)
- Acquisition functions


## Acquisition functions
Acquisition functions are functions of the input space $x$ parameterized by the 
- observed data $D$
- GP hyperparameters $\theta$ (for now, we do not consider this term for simplicity)

We use $\alpha(x;D)$ to denote an acquisition function. They should be cheap to evaluate. (Because the objective function $f(x)$ is expensive to evaluate)

Popular acquisition functions 
- probability of improvement (PI)
    - $\alpha_{PI}(x; D_{n}) \gets P(f(x)>\tau) = \Phi(\frac{\mu_{n}(x)-\tau}{\sigma_{n}(x)})$
- expected improvement (EI)
    - $\alpha_{EI}(x; D_{n}) \gets E[I(x,f(x),\theta)] = (\mu_{n}(x)-\tau)\Phi(\frac{\mu_{n}(x)-\tau}{\sigma_{n}(x)})+\sigma_{n}(x)\phi(\frac{\mu_{n}(x)-\tau}{\sigma_{n}(x)})$
- upper confidence bound (UCB) 
    - $\alpha_{UCB}(x; D_{n}) \gets \mu_{n}(x)+\beta_{n}\sigma_{n}(x)$
-  predictive entropy search (PES)

The following image shows the $\alpha_{EI}$ whose aim is to find the **mimimum** of $f(x)$.
<img src="AC_to_find_minimum.png">


## Steps to Optimize Function $f(x)$
1. Place a prior (using GP)
2. Construct an acquisition function to determine the next query point x
    $$ x_t = argmax_x \alpha(x|D_{t-1}) $$
3. Evaluate $f(x)$
4. Form a posterior distribution (using GP)
4. Augment data $D_{t} = \{ D_{t-1}, (x_{t},y_{t}) \}$
5. go to 2


## Conclusion
- ﬁnd the minimum (maximum) of difﬁcult non-convex functions with relatively **few evaluations**,<br/>
    at the cost of performing **more computation** to determine the next point to try.


## Example
This example is to find the **maximum** of the function $f(x)$.
<img src="iterations.png">


## References
(Asterisks represents the amount of contributions. The higher, the better. Mximum is 5.)
- https://github.com/krasserm/bayesian-machine-learning/blob/master/bayesian_optimization.ipynb $(*****)$
- https://github.com/krasserm/bayesian-machine-learning/blob/master/gaussian_processes.ipynb $(*****)$
- https://www.quora.com/How-does-Bayesian-optimization-work $(*****)$
- http://katbailey.github.io/post/gaussian-processes-for-dummies/ $(*****)$
- Shahriari, B., Swersky, K., Wang, Z., Adams, R. P., & De Freitas, N. (2016). Taking the human out of the loop: A review of bayesian optimization. Proceedings of the IEEE, 104(1), 148-175. $(****)$
- https://www.youtube.com/watch?v=-Ekte3my510 $(***)$
- https://towardsdatascience.com/shallow-understanding-on-bayesian-optimization-324b6c1f7083 $(*)$ 

---
# 2019-04-06

# SMBO and Bayesian Optimization

应该把SMBO和Bayesian optimization想象成一个**辅助**的工具。

# 想象一个寻宝的故事

现在我知道国内有许多品质不同的宝石(hyperparameter)，有烂石头、有带瑕疵的红宝石、蓝宝石、以及全世界最昂贵的一颗钻石（optimal hyperparameter）。

我现在想找到一颗宝石，把它卖掉换钱（performance），钱越多越好。

但我不知道它们在哪里（假设最宝贵的钻石在**海南省**）。中国有34个省级行政区，完全找遍一个地方需要花费**一天**的时间（完整地跑一次target function开销是巨大的）我只有**一周**的时间（budget是有限的）。

我有几种思路去找这个宝石。

### Random Search
给34个地方编个号，掷一个34面骰子，掷出几号我就去哪找。这一周的时间我就只能随便搜寻7个地方了，如果掷7次骰子都没选中海南，那我的时间用完了也找不到钻石。我可能能找到一些烂石头，也可能找到一些普通的水晶。

如果我掷骰子能掷到海南，那我就发财了。

### Grid Search
打开地图，从最左上角开始，往右往下找。这样，一周的时间我可以搜寻新疆、甘肃、内蒙古、青海、宁夏、山西、山西，7天时间用完了，我找到了一些黄金、水晶，但我没找到这个最好的钻石。

如果我选择grid search的思路是从最南到最北，那我也能很快找到这个最昂贵的钻石。

### SMBO / Bayesian Optimization
拿到了一个神奇的魔镜（SMBO / Bayesian Optimization），我告诉它我之前做了什么事情，它能以80%的把握感应到这颗钻石**可能**在哪里，它能指引我下一步往哪走。

最开始我什么都没做，因此用不上魔镜。我随便选一个地方，比如广东，花一天时间搜索，无结果。剩下6天。

我告诉魔镜，我搜过广东了，什么都没有。魔镜花半天时间思考后 **(SMBO / Bayesian Optimization本身也有开销)** 告诉我，“钻石很有可能在上海”。剩5.5天。

我花一天时间在上海搜寻，没结果，剩下4.5天。

我再找魔镜，说我找过广东和上海，都没结果。魔镜又花半天时间思考，告诉我“钻石有可能在云南”。剩4天。

一直重复这个过程......

最后到7天时间用完的时候，我只搜索了4个地方，第5个地方还没完全搜完（7/(1+0.5)=4.67）。这期间，魔镜**非常**有可能让我去海南找找，那我就可以找到了这个最宝贵的钻石。

这个魔镜**不能100%保证**我找到这个钻石，而且它思考的时候还要浪费我半天的时间，但还是愿意跟着它的指引，因为它能感应到钻石大致方位的这个超能力至少有80%的把握。如果是我自己去猜，那我猜中的可能性只有7/34，大概20%。


# 一些不清晰的地方
## 我的想法

魔镜不能直接凭空变出钱给我，也不能凭空变出最昂贵的钻石给我。它只能告诉我钻石可能在哪里，我还是要自己去找。
```
SMBO不能直接给让target model得到最优performance，
也不能直接告诉你最优的hyperparameter是什么。
SMBO只能说：“这个hyperparameter可能挺不错的，你去试试”
然后我们还是要跑一遍target model，
然后再去问SMBO，“接下来我该试试哪一个呢？”
```

魔镜到底靠什么来感应钻石所在的大概方位呢？我告诉魔镜我之前探索过哪些地方，结果如何，魔镜在思考的时候就会在脑子里利用我告诉它的信息，计算出一个全中国宝石概率分布图，然后找到概率最高的那个地方，让我去那个地方找找看。
```
那么SMBO到底用什么来猜哪个hyperparameter可能挺好呢？
在家焕学长发的slides的第8页的SMBO伪代码第4行右边有一个M，
这一页最上面有个解释说M是SMBO's model。
所以，你告诉SMBO你之前做过什么事情，结果怎么样，
SMBO就计算一个model M来告诉你，
接下来试一下哪个hyperparameter可能结果更好
```